In [1]:
import pandas as pd
import numpy as np
import h5py
import pyBigWig

In [18]:
genome = 'mm10'
window_kb = 5
bin_size = 10

genome = 'hg38'
window_kb = 5
bin_size = 10
my_tf='PRMT6'

# get promoterome
infile_promoterome=f"/home/jbreda/Promoterome/results/{genome}/promoterome_pm{window_kb}kb_filtered_clustered_sorted.bed"
promoterome = pd.read_csv(infile_promoterome,sep='\t')
N_prom = promoterome.shape[0]

# get one tf
infile_svd = f'../results/{genome}/svd/Window_pm{window_kb}kb_bin_size_{bin_size}/{my_tf}.hdf5'
with h5py.File(infile_svd, 'r') as hf:
    u = hf['U'][:,0]
    s = hf['S'][0]
    N_pos = hf['U'].shape[0]//N_prom

chip = np.reshape(u*s,[N_prom,N_pos]).T

# get chrom size and initialize chr_values
chrom_size_file = f"../resources/{genome}.chrom.sizes"
chr_size = []
chr_values = {}
CHR = promoterome.chr.unique()
with open(chrom_size_file,'r') as f:
    for l in f.readlines():
        l = l.strip()
        chr = 'chr'+l.split('\t')[0]
        size = int(l.split('\t')[1])
        if chr in CHR:
            chr_size.append((chr,size))
            chr_values[chr] = np.zeros(int(np.ceil(size/bin_size)))

# fill in chr_values with chip signal
for p in range(N_prom):

    # skip if all nan
    if np.all(np.isnan( chip[:,p] )):
        continue

    # skip if all zero
    if np.all( chip[:,p] == 0 ):
        continue

    # get absolute and relative idx
    chr = promoterome.at[p,'chr']
    start_idx = int(np.ceil(promoterome.at[p,'start']/bin_size))
    rel_idx =  np.where( ~np.isnan(chip[:,p]) )[0]
    idx = start_idx + rel_idx

    # add values
    chr_values[chr][idx] = chip[rel_idx,p]
    # if 2 promoters are overlaping the same bin, the chip signal is the same for both

# write bigwig
outfile = 'tmp_output.bw'
with pyBigWig.open(outfile, 'w') as bw:

    # add headers
    bw.addHeader(chr_size)

    # fill in the big wig chr by chr
    for chr in chr_values.keys():
        
        # get indices with non zero values
        idx = np.where( chr_values[chr] != 0 )[0]

        # get coordinates and values
        chrs = idx.shape[0]*[chr]
        starts = idx*bin_size
        ends = starts + bin_size
        values = chr_values[chr][idx]

        # add entries
        bw.addEntries(chrs, starts, ends, values)

RuntimeError: The entries you tried to add are out of order, precede already added entries, or otherwise use illegal values.
 Please correct this and try again.


In [14]:

len(idx)

0